# Data Carpentry

Up until this point, reading in our data has been relatively simple. There is a bit of a learning curve to learning a programming language, but our files have been organized in such a way that we have been able to start manipulating it upon reading it in. More often than not, our data will not be in this pre-packaged, ready to play with format. That's where data carpentry comes in.

Data carpentry, sometimes referred to as data cleaning, is the first step after acquiring a dataset as your data are rarely in a format that is ready for analysis. This will become more evident to you as you start exploring your own datasets that you will use to answer some type of question, or predict some outcome.

So what does "messy" data look like? Take a look at the image below...

![title](../images/messy2.png)

This is the data we will be working with today.

In [ ]:
# import the required packages

# xlrd is a package for developers to extract data from
# Excel spreadsheets. https://pypi.python.org/pypi/xlrd

import pandas as pd 
import xlrd 

### The Data

This is only a partial dataset on several [species of dessert rodents](http://esapubs.org/archive/ecol/E090/118/Portal_rodent_metadata.htm). The files that we have used before have all been `csv`s, but `Python` can handle a lot more types of file formats. The file we will be working with today is in `.xls` format, otherwise known as an Excel file. But when our data looks like it does above, how is `pandas` going to interpret this? Let's find out...

In [ ]:
# just view what the data would look like when read by pandas

pd.read_excel('../../../datasets/messy_survey.xls') 

Calling this "messy" is an understatement. Remember, we are essentially dealing with three different tables in the exact same file. You will notice all of the `NaN` values. This stands for "Not a Number" and is the default value for those cells that don't contain data. We will touch more on how to handle these within a dataset in a bit, but for now, we want to get rid of all of those columns and rows that are entirely `NaN`s, but first, let's actually assign this data frame object a name. We'll call it `file`.

In [ ]:
# this object is the same as above except
# now it is saved to the object `file`

file = pd.read_excel('../../../datasets/messy_survey.xls', header = None)

One thing to note here, we have yet to see the `header` parameter used. Normally, when a data frame object is created, the first row in the file is interpreted to be the header row, and often this is okay. But in its current state, our data is in no place to have a header row yet. Plus, the first row in the dataset for this Excel file is an empty line. When there aren't any data to be the header row, `pandas` defaults the names to a series of **`Unamed`**s (take a look at the data frame above).

Well, this row is unnecessary for the current state of the data, so we are going to specify the `header` as `None`.

Okay, so back to removing those columns filled entirely with `NaN`s. This will make cleaning up the data a bit easier if we just go ahead and remove these. There are several ways you could go about doing this. Take a look at one way it can be done...

In [ ]:
# for the sake of keeping organized, we are 
# going to call this modified data frame `messy`.
# You will see this several times throughout this 
# lesson, and is unnecessary, but it will allow us
# to keep track of edits to the original data frame

messy = file.ix[5:,2:].reset_index(drop = True)
messy.head(20)

We are introduced to some new methods in the code above, so let's talk about them so we have a better understanding of what they are doing for us.

`.ix` is an locator method based off of index. In other words, you can specify some index/indexes and it will return a subset of the data frame based on the index locations that you specified. In the above, we specify that we only want those rows starting from the 5th index and below (`file.ix[`**`5:`**`,2:]` (*bolded section*)). Then we also want to ignore the first 2 columns, which means that we are only interested in those columns starting from the 2nd index and beyond (`file.ix[5:,` **`2`**`:]` (*bolded section*)). Remember, `.ix` is a `pandas` method, so it must be called on a `pandas` object. In this case, we are using it on the `file` data frame that we created above. 

`.reset_index` is a method to do just that, reset the index. Now the reason we use it, is because we are removing some columns and rows, and without it, the new data frame, `messy` would begin at the row index of 5. This isn't a huge deal, bit it is more convenient if it starts where indexes normally start... at 0. Now, there is also a parameter that we call in this method, `drop` and we give it the argument, `True`. If we were to exclude this parameter, the new data frame would have a new column called "Index" with the old, non-reset index.

So this next section may be a little complicated, particularly if you are new to programming, but we will take this step-by-step.

In [ ]:
table_names = ["Species: DM", "Species: DO", "Species: DS"]
groups = messy[2].isin(table_names).cumsum()
tables = {g.iloc[0,0]: g.iloc[1:] for k,g in messy.groupby(groups)}

**Line 1:** The first thing we want to do is identify all of the individual tables that we will eventually recombine to make one data frame. We are going to do this by taking a name associated with each table\*. Take a look at the image of the Excel file above. Each table has a name associated with it referring the species of organism. This will be good for us to use as the table name. 

**Line 2:** The next thing we are going to want to do is group the data based on what table it belongs to. In this case, anything below that table name and before the next table name belongs to that table. Now it is just locating where those table names are. That is what the `.isin` method is doing, and since these values are in column **2**, we will go ahead and specify this (`groups =` **`messy[2].isin(table_names)`**`.cumsum()` (*bolded section*)). Then we pass the table_names list in the `.isin` method, and it will return either `True` or `False` depending on whether one of the specified table names is a value. What this returns is a `pandas` series object that has 3 `True` values, corresponding to the table name values, and the rest are `False`. To see what this returns, create a cell and run this line, `messy[2].isin(table_names)` after the next code cell. 

Now, the the next method, `.cumsum` is where it gets a bit creative for this particular dataset. `.cumsum` stands for "cumulative sum", and all it does is run through each item in a series and adds the previous number in the series to the current number it is on in the iteration. We call this on our `messy[2].isin(table_names)` series that we talked about above. Yes, this is a series of booleans, but in `Python` a `True` can be interpreted as `1` and `False` as `0`. Therefore, the first value in the series, which is `True`, will be 1. Then, since the next value is `False`, or 0, the second value will be 0 + 1, or 1. This will continue to happen until the iteration reaches a new `True` value, and then it becomes 2, which will delineate the next table. I know that this can be confusing, but play with the code a little bit to see what each component produces. This will help you if you do well with visualizing each part.

**Line 3:** This next line is just a one-liner for loop that creates what is known as a dictionary. Dictionaries in `Python` have a key and a value. In this case, we are going to want to turn the name of the table into the dictionary key, and the data of the tables as the values. Notice how we use groupby. This will allow the for loop to iterate of each table (as they are now grouped numerically together). We then use the `iloc` method to find the data that we wish to assign to both the key and the value of the dictionary. In this case, `iloc[0,0]` is the name of the table for each table, while `iloc[1:]` is all of the values of the table for each table. You can now reference the values of the "Species: DM" table by calling `tables['Species: DM']`.  

Again, if this all seems confusing, break it up and play with it in a cell to see what each piece returns. It may be simpler to visualize first, and then grasp the logic.

\* *This is a good time to say that not all messy data is going to be cleaned this way. What is important is to know how to work with messy data. This may take some creativity given each task, but in the long run, you will begin to build up your working vernacular, and patterns will begin to appear in which cleaning data will become a lot less abstract.*

#### What do these tables look like now?

Let's take a pause in our cleaning process, and print out what these separated tables look like now. We will create a for loop that iterates of each table, prints out it's name and then the values.

In [ ]:
list(tables)
for k,v in tables.items():
    print("table:", k)
    print(v)
    print()

Remember that each table is now a dictionary, so in the for loop we specify `for k,v in table.items():`. `k` stands for key, `v` for value (although this nomenclature is not necessary, just friendlier). This is how you iterate over dictionaries. We then print the `k` first, the table name, then the `v`, the data within the table. The last `print` statement just creates a space between one table and the next.

### Creating the dataframe

Now is the time to put the pieces back together. We really want to create a single data frame and create a new variable specifying the species. Take a look at how we use a for loop below to iterate over `table.items()` again.

In [ ]:
dfs = [] # an empty list where we will store our separate dataframes

for k,v in tables.items(): # iterate over the table dictionaries
    single_frame = pd.DataFrame(tables[k]) # create a dataframe from each table 
    single_frame['species'] = k # create a new column called species and fill it with the name of the data frame
    single_frame = single_frame.reset_index(drop=True).ix[1:] # reset index and remove first row (header row)
    single_frame.columns = ['date','plot','sex','weight','species'] # rename the columns
    dfs.append(single_frame) # add to the list of separate data frames

df = pd.concat(dfs).reset_index(drop = True) # join the dataframes together into one data frame
df # return the complete data frame    

There are a lot of steps in this for loop. For that reason, each line of code is commented to describe what each piece is doing. 

We are now in pretty good shape, but there are still some things we need to do before we can consider this tidy. One thing that should be noticeable right of the bat is there are rows of data that are almost completely `NaN` values, save for the `Species` column. Why is that? 

Let's take a look at the picture again...

![title](../images/messy2.png)

Notice that between each table there is a blank row. That is the very reason for these lines. Essentially, these are residual from our initial reading in of the file and they have remained throughout the process. Well, now it is time to get rid of them.

In [ ]:
df.dropna().reset_index(drop = True)

Hmm...so the original data frame had 39 rows (starting with an index at 0), but now it appears to only have 34. There were only two of these blank lines in the original file, so what happened? Well, it removed any row with an `NaN` value. But that wasn't our intention. We kind of want to keep those rows and I will explain why later.

Below is how you specify that if a row has an `NaN` value for a specific row, then remove that row. Let's say that if a row doesn't have a date, then we will remove that row.

In [ ]:
# the frame is now in a state that could be
# considered workable. We will call this workable
# data frame `df`.

df = df.dropna(subset = ['date']).reset_index(drop = True)
df

That's better; 37 rows are what we should see. 

## Column Clean Up

Now we can start looking into the columns a bit. Take the values in `species` column for example. Having "Species:" repeated for every single value is unnecessary. We are going to go ahead and remove this segment, leaving only the species abbreviations.

Again, there are several ways to do this, but the method below takes on a familiar format that we have seen before. 

In [ ]:
abbrv = []
for i in df.species:
    abbrv.append(i.split(" ").pop())
    
df['species'] = abbrv

df.head()

That looks a lot better. Let's talk about how we did it.

**line 1** `abbrv = []` -> this creates an empty list where we will store our modified values.

**line 2** `for i in df.species:` -> the beginning of the for loop. For each item in the `df['species']` column.

**line 3** `abbrv.append(i.split(" ").pop())` -> split the string up by words, then pop, which pops the last index off, in this case the species abbreviation, and returns it. We then add that to the list `abbrv`.

**line 4** `df['species'] = abbrv` -> assign the new values to the `df['species']` column.

Let's take a look at the `df['weight']` column. This is a variable that we would like to find some stats about, but in its current state we can't do that. Take a look at the column again. Some of the values have a "g" added to the number indicating grams. It is this "g" that we want to remove so that we can start running some stats on it. Below is one way to do that.

In [ ]:
# for this we need to import the numpy package
 
import numpy as np

nums = []
for i in df.weight:
    if pd.isnull(i):
        val = np.nan
    else:
        val = ''.join(c for c in str(i) if c.isdigit())
    nums.append(val)
df['weight'] = nums


The method is similar to how we modified the `species` column except we added a conditional. The reason we need the conditional is because `NaN` values will halt our for loop and we don't want to get rid of them, so we first need to check if the value is `NaN`. If the values are and `NaN`, we will return them as `NaN`, otherwise, we use this line of code to take only those values that are numbers and return them.

But there is one problem... our column is still a string type, and you can't call np.mean() on a string. So let's convert it to a float.

In [ ]:
df['weight'] = df['weight'].astype(float)

In [ ]:
df.weight

Now what to do about those pesky `NaN`s? Well, we could remove rows with those values, or we can fill them in. Below is how you fill them in using the mean weight, this way we are using the average of the dataset.

But let's take a look at the column again. We will copy the data frame to a frame called `cleaned`.

In [ ]:
cleaned = df
cleaned['weight']

In [ ]:
cleaned['weight'] = cleaned['weight'].fillna(cleaned['weight'].mean())

And that is how you would fill in the `NaN` values with the mean weight. This allows us to keep the information of the other columns, which we may be useful, but not affect the overall mean of the weigth column. Take a look at our final frame below.

**To think about:** Is there a better way to fill in these `NaN`s? 

In [ ]:
cleaned

Well,that does it for this lab. There was a lot, and we were introduced to some new methods. Remember, it isn't necessary for you to memorize each method. Instead, focus on what needs to be done in order to make a dataset clean. From there, we can figure out methods that will get the job done with some seaching and a little ingenuity. In the practice, we will dive a bit deeper into these methods...